In [ ]:
pip install nltk

In [ ]:
#Importamos librerías
import pandas as pd
import numpy as np
import re
import nltk
import seaborn as sns
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_samples, silhouette_score




# Descargar recursos necesarios
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
# Leemos el archivo CSV llamado "playlist_lyrics_limpio.csv" y guardamos en el DataFrame df
df = pd.read_csv("playlist_lyrics_limpio.csv")

In [ ]:
df

In [ ]:
# Definimos una función para limpiar texto eliminando HTML, signos de puntuación y espacios extra
def limpieza_basica(df):
    if pd.isnull(df): # Si el valor es nulo, devuelve una cadena vacía
        return ""
    texto = df.lower()                                # Convierte todo el texto a minúsculas
    texto = re.sub(r'<.*?>', ' ', texto)              # Eliminamos etiquetas HTML
    texto = re.sub(r"[^a-z\s]", " ", texto)           # Eliminamos todo lo que no sea letras o espacios
    texto = re.sub(r'\s+', ' ', texto).strip()        # Reemplazamos múltiples espacios por uno solo y elimina espacios iniciales/finales
    return texto

# Aplica la limpieza de texto a la columna de letras de canciones para prepararla para el análisis
df['letra_limpia'] = df['lyrics'].apply(limpieza_basica)

In [ ]:
# Creamos una columna con los tokens
df['tokens'] = df['letra_limpia'].apply(lambda x: x.split())

In [ ]:
# Definimos una función que elimina las stopwords de una lista de palabras
def quitar_stopwords(palabras):
    return [p for p in palabras if p not in ENGLISH_STOP_WORDS]

# Aplicamos la función para quitar stopwords a la columna de tokens y guarda el resultado en una nueva columna
df['letra_sin_stopwords'] = df['tokens'].apply(quitar_stopwords)

In [ ]:
# Inicializamos el lematizador de NLTK (WordNet)
lemmatizer = WordNetLemmatizer()

# Función de lematización que recibe una lista y devuelve una lista, aplica lematización a cada palabra
def solo_lemmatize(palabras):
    if isinstance(palabras, list):  # Verifica que sea una lista
        return [lemmatizer.lemmatize(p) for p in palabras]  # Lematiza cada palabra
    else:
        return []

# Aplicamos la lematización a las letras sin stopwords y guarda el resultado en una nueva columna
df['letra_lema'] = df['letra_sin_stopwords'].apply(solo_lemmatize)

In [ ]:
letra_final = df['letra_lema'].apply(lambda x: " ".join(x))  # Convertimos la lista de palabras lematizadas en un solo string por canción
vectorizer = TfidfVectorizer()      # Inicializa el vectorizador TF-IDF
X_tfidf = vectorizer.fit_transform(letra_final)     # Ajusta y transforma el texto en una matriz de TF-IDF

In [ ]:
# Aplicamos K-Means para agrupar las canciones en 4 clusters basados en sus letras vectorizadas
kmeans = KMeans(n_clusters=4, random_state=42)
df['cluster'] = kmeans.fit_predict(X_tfidf)

# Muostramos cuántas canciones hay en cada cluster
print(df['cluster'].value_counts())

# Mostramos ejemplos de canciones por cada cluster
for i in range(4):
    print(f"\n Cluster {i} - Ejemplos:")
    print(df[df['cluster'] == i]['title'].head(7))  # Muestra los primeros 7 títulos del cluster i

In [ ]:
# Calcula las coordenadas de los centroides de los 4 clusters
centroids = kmeans.cluster_centers_

# Calcula una matriz de distancias euclidianas entre cada par de centroides
dist_matrix = np.linalg.norm(centroids[:, np.newaxis] - centroids, axis=2)

# Genera un heatmap que muestra gráficamente las distancias entre clusters
plt.figure(figsize=(6, 5))
sns.heatmap(dist_matrix, annot=True, cmap="YlGnBu", xticklabels=[f'C{i}' for i in range(4)], yticklabels=[f'C{i}' for i in range(4)])
plt.title("Distancia entre Clusters (centroides)")
plt.show()

In [ ]:
# Calculamos los valores individuales de Silhouette para cada punto (qué tan bien está asignado al cluster)
silhouette_vals = silhouette_samples(X_tfidf, df['cluster'])
df['silhouette'] = silhouette_vals    # Guarda los valores en el DataFrame

# Número total de clusters detectados
n_clusters = df['cluster'].nunique()

# Creaamos el gráfico de Silhouette por cluster
fig, ax = plt.subplots(figsize=(10, 6))

y_lower = 10  # Posición inicial vertical (eje y)
for i in range(n_clusters):
    # Extraemos y ordenamos los valores de silhouette del cluster actual
    cluster_vals = silhouette_vals[df['cluster'] == i]
    cluster_vals.sort()

    size_cluster_i = len(cluster_vals)
    y_upper = y_lower + size_cluster_i

    # Asignamos un color único al cluster y dibuja su área
    color = plt.cm.nipy_spectral(float(i) / n_clusters)
    ax.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_vals,
                     facecolor=color, edgecolor=color, alpha=0.7)

    # Agregamos la etiqueta del cluster
    ax.text(-0.05, y_lower + 0.5 * size_cluster_i, f'Cluster {i}')

    y_lower = y_upper + 10  # espacio entre clusters

# Línea roja vertical indicando el promedio global de Silhouette
sil_avg = silhouette_score(X_tfidf, df['cluster'])
ax.axvline(sil_avg, color="red", linestyle="--", label=f'Media global: {sil_avg:.2f}')

# Ajustes del gráfico
ax.set_title("Silhouette Plot por Cluster", fontsize=14)
ax.set_xlabel("Silhouette Coefficient")
ax.set_ylabel("Puntos de muestra")
ax.set_xlim([-0.1, 1])
ax.set_yticks([])
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Obtenemos el listado de todas las palabras usadas en el vectorizador TF-IDF
palabras = vectorizer.get_feature_names_out()

# Obtenemos los centroides del modelo KMeans
centroides = kmeans.cluster_centers_

# Mostramos las top 10 palabras más importantes por cluster
top_n = 10

for i, centroide in enumerate(centroides):
    print(f"\n Cluster {i} - Palabras clave:")
    # Ordenamos los índices de las palabras según su peso en el centroide, de mayor a menor
    top_indices = centroide.argsort()[::-1][:top_n]
    # Obtenemos las palabras correspondientes a esos índices
    top_palabras = [palabras[ind] for ind in top_indices]
    # Imprime las palabras clave separadas por coma
    print(", ".join(top_palabras))
